In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp crossing

In [ ]:
from chewc.core import *
from chewc.meiosis import *
import torch

## Crossing
> Using gamete operations to produce new individuals

In [ ]:
#| export
from chewc.core import *
from chewc.meiosis import *
from chewc.trait import *
import torch
def double_haploid_operation(gamete):
    return gamete.repeat(2, 1, 1)

def make_DH(genetic_map, individual):
    haploid_line = simulate_gametes(genetic_map, individual)
    return double_haploid_operation(haploid_line)

In [ ]:
#| export

def cross_operation(egg, pollen):
    return torch.stack((egg, pollen), dim=0).squeeze(1)

def make_cross(genetic_map, mother, father):
    egg = simulate_gametes(genetic_map, mother)  # Simulating gametes for the mother
    pollen = simulate_gametes(genetic_map, father)  # Simulating gametes for the father
    return cross_operation(egg, pollen)

In [ ]:
#| export
def random_crosses(population: Population, 
                   n_crosses: int, 
                   ) -> Population:
    """
    Performs random crosses within a population.

    Args:
    ----
        population (Population): The population to perform crosses within.
        n_crosses (int): The number of crosses to perform.

    Returns:
    -------
        Population: A new population of offspring resulting from the crosses.
    """
    # make sure there are enough individuals for the requested number of crosses.
    if n_crosses > len(population.individuals) // 2:
        raise ValueError(f"Not enough individuals in the population ({len(population.individuals)}) for {n_crosses} crosses. Need at least {2*n_crosses}.")

    # sample parents randomly
    selected_parents = random.sample(population.individuals, 2 * n_crosses)

    # create offspring
    offspring = []
    for i in range(n_crosses):
        mother = selected_parents[i * 2]
        father = selected_parents[i * 2 + 1]
        haplotypes = make_cross(population.individuals[0].genome.genetic_map, mother, father)
#         genetic_value = trait.calculate_genetic_value(haplotypes.unsqueeze(0)).squeeze(0)  # calculate GV
        offspring.append(
            Individual(
                genome = mother.genome,
                haplotypes=haplotypes,
                mother_id=mother.id,
                father_id=father.id,
#                 genetic_value=genetic_value,
            )
        )
    return Population(individuals=offspring)

In [ ]:
#| export
def random_crosses(parent_pop:Population, n_crosses,genetic_map)-> Population:
    new_pop = []
    total_cross=n_crosses
    random_pairs = torch.randint(0,len(parent_pop), (total_cross*2,)).reshape(-1,2)
    cross_pairs = parent_pop[random_pairs]
    for pair_idx in range(len((random_pairs))):
        egg = cross_pairs[pair_idx][0] 
        pollen = cross_pairs[pair_idx][1]
        x = make_cross(genetic_map, egg , pollen)
        new_pop.append(x)
    new_pop = torch.stack(new_pop)
    return new_pop

In [ ]:
#| export
def truncate_select(tgv):
    return torch.topk(tgv,20).indices

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()